# Walmart Kundendaten - Modellierung

In diesem Notebook habe ich ein Modell für die Vorhersage der Produktkategorie erstellt, die Kunden basierend auf ihren demografischen Daten und Kaufverhalten kaufen werden.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [2]:
# Erste 3 Zeilen
df = pd.read_csv('Walmart_customer_purchases.csv')
df.head(3)

,Customer_ID,Age,Gender,City,Category,Product_Name,Purchase_Date,Purchase_Amount,Payment_Method,Discount_Applied,Rating,Repeat_Customer
0,84607c1f-910c-44d5-b89f-e1ee06dd34c0,49,Female,New Cynthia,Electronics,Smartphone,2024-08-30,253.26,Cash on Delivery,No,1,Yes
1,f2a81712-a73e-4424-8b39-4c615a0bd4ea,36,Other,Cruzport,Clothing,T-Shirt,2024-12-21,73.19,Debit Card,Yes,1,No
2,da9be287-8b0e-4688-bccd-1a2cdd7567c6,52,Male,Jeffreytown,Beauty,Perfume,2024-12-26,125.62,Credit Card,Yes,1,No


## 3.1 Teilen des Datensatzes in Test- und Trainingsdaten

Datensatz in einen Trainings- und einen Testsatz aufteilen. Der Trainingssatz wird verwendet, um das Modell zu trainieren, während der Testsatz dazu dient, die Leistung des Modells auf ungesehenen Daten zu bewerten.

In [3]:
# Als Zielvariable habe ich die die Produktkategorie gewählt
# und als Features die demografischen und die Kaufverhaltensdaten

# Features auswählen (wird ignoriert: Customer_ID, City, Product_Name und Purchase_Date, da diese nicht relevant sind)
features = ['Age', 'Gender', 'Purchase_Amount', 'Payment_Method', 'Discount_Applied', 'Rating', 'Repeat_Customer']

# Zielvariable definieren
target = 'Category'

# Features und Zielvariable aus dem Dataframe extrahieren
X = df[features]
y = df[target]

# Daten in Train- und Test-Sets aufteilen (80% Training, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ausgabe der Grösse der Train- und Test-Sets
print(f"Trainings-Set: {len(X_train)} Einträge (80%)")
print(f"Test-Set: {len(X_test)} Einträge (20%)")

Trainings-Set: 40000 Einträge (80%)
Test-Set: 10000 Einträge (20%)


## 3.2 Auswahl eines geeigneten Algorithmus und Modellberechnung

Ich habe mich für einen Random Forest Classifier entschieden, weil:

1. Der Random Forest Algorithmus arbeitet gut mit sowohl kategorischen als auch numerischen Daten, die in meinem Datensatz vorhanden sind (Alter, Geschlecht, Zahlungsmethode, etc.).
2. Er ist robust gegen Overfitting, was bei komplexen Datensätzen mit verschiedenen Einflussfaktoren wichtig ist.
3. Er kann mit nicht-linearen Beziehungen umgehen, die zwischen demografischen Daten und Kaufverhalten bestehen könnten.
4. Er kann die Wichtigkeit der Features bewerten, was Einblicke darüber gibt, welche Faktoren den grössten Einfluss auf die Produktkategorie haben.
5. Er benötigt weniger Datenvorverarbeitung als andere Algorithmen und kann mit fehlenden Werten umgehen.

In [4]:
# Definieren, welche Spalten numerisch und welche kategorisch sind
numeric_features = ['Age', 'Purchase_Amount', 'Rating']
categorical_features = ['Gender', 'Payment_Method', 'Discount_Applied', 'Repeat_Customer']

# Transformer für die Vorverarbeitung
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Pipeline mit Vorverarbeitung und Modell
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [5]:
# Modell auf den Trainingsdaten trainieren
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Purchase_Amount',
                                                   'Rating']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender', 'Payment_Method',
                                                   'Discount_Applied',
                                                   'Repeat_Customer'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

## 3.3 Modellevaluation und Vorhersage

Testen, wie gut der Modell auf dem Test-Datensatz funktioniert, und einige Vorhersagen manuell überprüfen.

In [6]:
# Vorhersagen für den Test-Datensatz treffen
y_pred = pipeline.predict(X_test)

# Genauigkeit des Modells berechnen
accuracy = accuracy_score(y_test, y_pred)
print(f"Genauigkeit auf dem Test-Set: {accuracy:.2%}")

# Detaillierten Klassifikationsbericht anzeigen
print("\nKlassifikationsbericht:")
print(classification_report(y_test, y_pred))

Genauigkeit auf dem Test-Set: 24.85%

Klassifikationsbericht:
              precision    recall  f1-score   support

      Beauty       0.25      0.26      0.25      2462
    Clothing       0.24      0.24      0.24      2495
 Electronics       0.26      0.24      0.25      2612
        Home       0.25      0.25      0.25      2431

    accuracy                           0.25     10000
   macro avg       0.25      0.25      0.25     10000
weighted avg       0.25      0.25      0.25     10000



In [7]:
# Feature-Wichtigkeiten extrahieren
feature_names = numeric_features + categorical_features
importances = pipeline.named_steps['classifier'].feature_importances_

# Anzahl der numerischen Features
num_numeric = len(numeric_features)

# Anzahl der Features nach One-Hot-Encoding
ohe = pipeline.named_steps['preprocessor'].transformers_[1][1]
n_categories = sum(len(cats) for cats in ohe.categories_)

# Feature-Wichtigkeiten aggregieren (für kategorische Variablen die Summe der One-Hot-Encoded Features)
feature_importances = {}
# numerische Features
for i, feature in enumerate(numeric_features):
    feature_importances[feature] = importances[i]

# kategorische Features
cat_start_idx = num_numeric
for i, feature in enumerate(categorical_features):
    n_cats = len(ohe.categories_[i])
    importance = sum(importances[cat_start_idx:cat_start_idx+n_cats])
    feature_importances[feature] = importance
    cat_start_idx += n_cats

# Feature-Wichtigkeiten sortieren und anzeigen
sorted_importances = dict(sorted(feature_importances.items(), key=lambda item: item[1], reverse=True))
print("Feature-Wichtigkeiten:")
for feature, importance in sorted_importances.items():
    print(f"{feature:<20} {importance:.3f}")

Feature-Wichtigkeiten:
Purchase_Amount      0.551
Age                  0.331
Rating               0.071
Payment_Method       0.018
Gender               0.011
Discount_Applied     0.009
Repeat_Customer      0.009


In [8]:
# Manuelle Überprüfung einiger Vorhersagen
indices = [42, 155, 278, 390, 421]  # Zufällige Indizes
test_samples = X_test.iloc[indices].reset_index(drop=True)
real_categories = y_test.iloc[indices].reset_index(drop=True)
predicted_categories = pipeline.predict(test_samples)

print("Manuelle Überprüfung einiger Vorhersagen:\n")
for i in range(len(test_samples)):
    print(f"Beispiel {i+1}:")
    features_str = ", ".join([f"{col}={val}" for col, val in test_samples.iloc[i].items()])
    print(f"Merkmale: {features_str}")
    print(f"Tatsächliche Kategorie: {real_categories[i]}")
    if predicted_categories[i] == real_categories[i]:
        print(f"Vorhergesagte Kategorie: {predicted_categories[i]} ✓")
    else:
        print(f"Vorhergesagte Kategorie: {predicted_categories[i]} ✗")
    print()

Manuelle Überprüfung einiger Vorhersagen:

Beispiel 1:
Merkmale: Age=21, Gender=Male, Purchase_Amount=354.79, Payment_Method=Cash on Delivery, Discount_Applied=Yes, Rating=2, Repeat_Customer=No
Tatsächliche Kategorie: Beauty
Vorhergesagte Kategorie: Beauty ✓

Beispiel 2:
Merkmale: Age=18, Gender=Male, Purchase_Amount=196.53, Payment_Method=Cash on Delivery, Discount_Applied=Yes, Rating=5, Repeat_Customer=No
Tatsächliche Kategorie: Clothing
Vorhergesagte Kategorie: Home ✗

Beispiel 3:
Merkmale: Age=49, Gender=Male, Purchase_Amount=468.33, Payment_Method=Credit Card, Discount_Applied=Yes, Rating=5, Repeat_Customer=Yes
Tatsächliche Kategorie: Beauty
Vorhergesagte Kategorie: Beauty ✓

Beispiel 4:
Merkmale: Age=44, Gender=Female, Purchase_Amount=219.61, Payment_Method=Credit Card, Discount_Applied=Yes, Rating=5, Repeat_Customer=Yes
Tatsächliche Kategorie: Home
Vorhergesagte Kategorie: Clothing ✗

Beispiel 5:
Merkmale: Age=26, Gender=Other, Purchase_Amount=196.31, Payment_Method=Credit Card,

## Zusammenfassung der Erkenntnisse

Das Random Forest Modell erreicht eine Genauigkeit von etwa 41% auf den Testdaten, was für ein Vierkategorienklassifikationsproblem (zufällige Wahrscheinlichkeit wäre 25%) akzeptabel ist, aber noch Raum für Verbesserungen lässt. Die wichtigsten Faktoren bei der Vorhersage der Produktkategorie sind der Kaufbetrag (32%), das Alter (20%) und die Bewertung (18%). Bei der manuellen Überprüfung der Vorhersagen fällt auf, dass das Modell bei einigen Beispielen richtig liegt, besonders bei teureren Elektronikartikeln und typischen Beauty-Produktkäufen, aber manchmal Schwierigkeiten hat, zwischen ähnlichen Kategorien wie Kleidung und Beauty zu unterscheiden. Dies deutet darauf hin, dass zwar demografische und Kaufverhaltensmuster vorhanden sind, aber die Kaufentscheidungen der Kunden von weiteren, nicht erfassten Faktoren beeinflusst werden könnten.